### 라이브러리

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_openml

In [2]:
# pytorch 라이브러리
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# gpu 사용 (mac m1은 mps)
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
print(f"gpu device: {device}")
print("torch.__version__:", torch.__version__)

gpu device: cpu
torch.__version__: 2.0.1+cu118


### 함수 정의

In [3]:
# 데이터 전처리
class custom_Dataset(Dataset):
    def __init__(self, image, label):
        self.image = image
        self.label = label

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        x = self.image[idx]
        y = self.label[idx]

        return x, y

In [4]:
# 모델 정의
class custom_TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(custom_TwoLayerNet, self).__init__()

        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        out = self.fc2(x)
        return out

### 메인 코드

In [6]:
# 데이터 수집
mnist = fetch_openml('mnist_784')
x_all, y_all = mnist.data, mnist.target
print(x_all.shape, y_all.shape)

(70000, 784) (70000,)


In [15]:
# 학습 및 검증 데이터 준비
y = pd.get_dummies(y_all) # 원 핫 인코딩 형태로 변환
x = x_all / 255. # 0에서 1로 정규화

x, y = torch.Tensor(np.array(x, dtype='float32')), torch.Tensor(np.array(y, dtype='float32'))

x_train, y_train = x[:60000], y[:60000]
x_test, y_test = x[60000:], y[60000:]

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

torch.Size([60000, 784]) torch.Size([60000, 10]) torch.Size([10000, 784]) torch.Size([10000, 10])


In [16]:
# 학습 및 검증 데이터 준비 (Pytorch)
batch_size = 32

train_dataset = custom_Dataset(x_train, y_train)
test_dataset = custom_Dataset(x_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [17]:
a = next(iter(train_loader))

In [18]:
# 모델, 손실함수, 최적화 방법 정의
model = custom_TwoLayerNet(input_size=x_train.shape[1], hidden_size=50, output_size=y_train.shape[1]).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [35]:
# 하이퍼 파라미터 지정
epochs = 10

train_loss_list = []
train_acc_list = []

In [37]:
# 학습 진행

# train
model.train()
for epoch in range(epochs):
    avg_loss = 0
    avg_acc = 0
    for data in train_loader:
        images = data[0].to(device)
        labels = data[1].to(device)

        optimizer.zero_grad() # 모델의 초기 gradient를 0으로 설정
        y_hat = model(images) # 모델의 forward를 통해 예측

        loss = criterion(y_hat, labels) # 자체적으로 softmax를 취하고 난 후 손실함수를 계산함
        loss.backward() # 손실함수에 대한 역전파 준비
        optimizer.step() # 가중치 업데이트

        avg_loss += loss
        avg_acc += sum(y_hat.max(axis=1).indices == labels.max(axis=1).indices)

    avg_loss = avg_loss / len(train_loader)
    avg_acc = avg_acc / (len(train_loader) * batch_size)

    train_loss_list.append(avg_loss.item())
    train_acc_list.append(avg_acc.item())

    print(f'epocs: {epoch+1}, loss: {avg_loss}, train_acc: {avg_acc}')

print('-'*100)
# test
model.eval()
with torch.no_grad():
    avg_acc_test = 0
    for data in test_loader:
        images = data[0].to(device)
        labels = data[1].to(device)

        pred = model(images)
        loss = criterion(pred, labels)

        avg_acc_test += sum(pred.max(axis=1).indices == labels.max(axis=1).indices)

    avg_acc_test = avg_acc_test / (len(test_loader) * batch_size)
    print(f'test_acc: {avg_acc_test}')

epocs: 1, loss: 0.08566725999116898, train_acc: 0.9813500046730042
epocs: 2, loss: 0.072810098528862, train_acc: 0.9835500121116638
epocs: 3, loss: 0.07923144847154617, train_acc: 0.9826333522796631
epocs: 4, loss: 0.08051019161939621, train_acc: 0.9827166795730591
epocs: 5, loss: 0.07776031643152237, train_acc: 0.983299970626831
epocs: 6, loss: 0.07418128103017807, train_acc: 0.9840333461761475
epocs: 7, loss: 0.08176115900278091, train_acc: 0.9836166501045227
epocs: 8, loss: 0.07722781598567963, train_acc: 0.9840999841690063
epocs: 9, loss: 0.06561639904975891, train_acc: 0.9850999712944031
epocs: 10, loss: 0.07991518825292587, train_acc: 0.98416668176651
----------------------------------------------------------------------------------------------------
test_acc: 0.9586661458015442
